In [1]:

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, TrainerCallback
from peft import LoraConfig, TaskType, get_peft_model
import torch
import psutil
import numpy as np



2025-06-10 16:02:23.293839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749571343.756630      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749571343.871596      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 6.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidi

In [3]:
torch.cuda.set_per_process_memory_fraction(1.0, 0)  # Use maximum available memory
torch.cuda.memory_max_split_size_mb = 64  # Set the max split size to avoid fragmentation

In [4]:
def print_memory_footprint():
    # GPU memory usage
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.memory_allocated() / (1024 ** 3)  # Convert to GB
        gpu_memory_cached = torch.cuda.memory_reserved() / (1024 ** 3)  # Cached memory
        print(f"[GPU] Memory Allocated: {gpu_memory:.2f} GB, Cached: {gpu_memory_cached:.2f} GB")
    else:
        print("[GPU] No GPU detected.")

    # CPU memory usage
    memory = psutil.virtual_memory()
    used_memory_gb = memory.used / (1024 ** 3)  # Convert to GB
    total_memory_gb = memory.total / (1024 ** 3)
    print(f"[CPU] Memory Usage: {used_memory_gb:.2f} GB / {total_memory_gb:.2f} GB")

In [5]:
'''
print("First example of blended_skill_talk training set:")
print(dataset['train'][0])
'''

'\nprint("First example of blended_skill_talk training set:")\nprint(dataset[\'train\'][0])\n'

In [6]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token, so we use eos_token

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [7]:
# Load dataset and tokenizer
dataset = load_dataset("allenai/soda")



def tokenize_function(examples):
    # Concatenate dialog turns into a single string for language modeling
    texts = [" ".join(dialog) for dialog in examples["dialogue"]]
    return tokenizer(texts, truncation=True, max_length=512)

# Tokenize datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'])
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(500))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(500))

README.md:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/689M [00:00<?, ?B/s]

valid.parquet:   0%|          | 0.00/82.9M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/84.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1191582 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/146346 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/148968 [00:00<?, ? examples/s]

Map:   0%|          | 0/1191582 [00:00<?, ? examples/s]

Map:   0%|          | 0/146346 [00:00<?, ? examples/s]

Map:   0%|          | 0/148968 [00:00<?, ? examples/s]

In [8]:
print(dataset)
tokenized_datasets


DatasetDict({
    train: Dataset({
        features: ['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'],
        num_rows: 1191582
    })
    validation: Dataset({
        features: ['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'],
        num_rows: 146346
    })
    test: Dataset({
        features: ['head', 'relation', 'tail', 'literal', 'narrative', 'dialogue', 'speakers', 'PersonX', 'PersonY', 'PersonZ', 'original_index', 'split', 'head_answer', 'pmi_head_answer', 'relation_tail_answer', 'pmi_relation_tail_answer'],
        num_rows: 148968
    })
})


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1191582
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 146346
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 148968
    })
})

In [9]:
# LoRA configuration for causal language modeling
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=1,
    lora_alpha=32,
    lora_dropout=0.09,
    target_modules=["q_proj"],
)

# %%

In [10]:
# Dynamic device assignment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained GPT-2 language model
# Load pre-trained GPT-2 model
#model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(device)
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                                             torch_dtype=torch.bfloat16,).to(device)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Enable gradient checkpointing if you run into memory issues
#model.gradient_checkpointing_enable()

# %%
# Print the model's architecture to inspect the names of the modules
#print(model)

# %%
print_memory_footprint()

Using device: cuda


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

[GPU] Memory Allocated: 2.05 GB, Cached: 2.14 GB
[CPU] Memory Usage: 2.43 GB / 31.35 GB


In [11]:
# %%

# Data collator for language modeling (masks tokens for prediction)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) # mlm=False for causal LM

# Perplexity as metric
#perplexity_metric = evaluate.load("perplexity", module_type="metric")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    shift_logits = torch.tensor(logits[:, :-1, :])
    shift_labels = torch.tensor(labels[:, 1:])
    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-100)
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    ppl = torch.exp(loss).item()
    return {"perplexity": ppl}

# Training args
training_args = TrainingArguments(
    output_dir="./lama-dialog-finetuned",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=100,
    #evaluation_strategy="epoch",
    #save_strategy="epoch",
    report_to="none",
    remove_unused_columns=False,
    bf16=True,  # Enable bfloat16
    fp16=False,  # Disable fp16 to avoid conflicts
)
# %%
print_memory_footprint()

[GPU] Memory Allocated: 2.05 GB, Cached: 2.14 GB
[CPU] Memory Usage: 2.44 GB / 31.35 GB


In [12]:
class MemoryCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        print("\nMemory footprint after evaluation:")
        print_memory_footprint()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    #eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
    callbacks=[MemoryCallback()]
)


/tmp/ipykernel_35/3187942852.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [13]:
any(p.requires_grad for p in model.parameters())

True

In [14]:
trainer.train()

Step,Training Loss
100,0.803200
200,0.775800


TrainOutput(global_step=250, training_loss=0.7808976593017578, metrics={'train_runtime': 126.8848, 'train_samples_per_second': 3.941, 'train_steps_per_second': 1.97, 'total_flos': 636044645670912.0, 'train_loss': 0.7808976593017578, 'epoch': 1.0})

In [15]:
trainer.save_model('TinyLlama-new-1000')

In [16]:
print_memory_footprint()

[GPU] Memory Allocated: 2.07 GB, Cached: 6.29 GB
[CPU] Memory Usage: 3.00 GB / 31.35 GB


In [17]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
smoothie = SmoothingFunction().method4
# Dynamic device assignment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# For generation, we'll load the model and use the `generate` method
model_for_generation = AutoModelForCausalLM.from_pretrained('TinyLlama-new-1000').to(device)
#model_for_generation = get_peft_model(model_for_generation, lora_config) # Ensure LoRA is applied

input_texts = [
    "So light is made up of colors?",
    "Why is the sky blue?",
    "How does gravity work?",
    "What causes rainbows?",
    "Why do we see lightning before thunder?",
    "How do plants make food?",
    "Why do we sleep?",
    "What is photosynthesis?",
    "How does the internet work?",
    "Why do birds migrate?"
]

reference_answers = [
    "Yes, light is made up of different colors. When they combine, they appear white. This can be separated with prisms, for example.",
    "The sky appears blue due to a phenomenon called Rayleigh scattering. Shorter blue wavelengths scatter more than red ones.",
    "Gravity is a force that attracts objects with mass toward each other. Earth pulls objects towards its center due to gravity.",
    "Rainbows occur when sunlight is both refracted and reflected inside raindrops, separating the light into different colors.",
    "Light travels faster than sound, so we see lightning before we hear thunder.",
    "Plants make food through photosynthesis, using sunlight, water, and carbon dioxide to produce energy and oxygen.",
    "We sleep to allow the body and brain to rest, recover, and process information from the day.",
    "Photosynthesis is the process plants use to convert sunlight into energy using chlorophyll, water, and carbon dioxide.",
    "The internet is a global network of computers that communicate via protocols like TCP/IP to exchange data.",
    "Birds migrate to find better food sources, breeding grounds, or climates that are more suitable during different seasons."
]

bleu_scores = []

for i in range(10):
    input_text = input_texts[i]
    reference = reference_answers[i]

    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    output = model_for_generation.generate(
        input_ids,
        max_length=70,
        num_return_sequences=1,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    candidate = generated_response.split()
    reference_tokens = [reference.split()]

    score = sentence_bleu(reference_tokens, candidate, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
    bleu_scores.append(score)

    print(f"\nPrompt {i+1}: {input_text}")
    print(f"Generated: {generated_response}")
    print(f"Reference: {reference}")
    print(f"BLEU-4 Score: {score:.4f}")

print("\nAverage BLEU-4 Score across all prompts: %.4f" % (sum(bleu_scores) / len(bleu_scores)))


Using device: cuda


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Prompt 1: So light is made up of colors?
Generated: So light is made up of colors?
Reference: Yes, light is made up of different colors. When they combine, they appear white. This can be separated with prisms, for example.
BLEU-4 Score: 0.0721

Prompt 2: Why is the sky blue?
Generated: Why is the sky blue?

1. The sky is blue because it reflects the sun's rays.

2. The sun's rays are blue because they have a high frequency of light.

3. The blue color of the sky is caused by the reflection of the sun's rays.
Reference: The sky appears blue due to a phenomenon called Rayleigh scattering. Shorter blue wavelengths scatter more than red ones.
BLEU-4 Score: 0.0175

Prompt 3: How does gravity work?
Generated: How does gravity work?
Reference: Gravity is a force that attracts objects with mass toward each other. Earth pulls objects towards its center due to gravity.
BLEU-4 Score: 0.0000

Prompt 4: What causes rainbows?
Generated: What causes rainbows?
Reference: Rainbows occur when sunlight 

# DPO

In [18]:
# train_dpo.py
import os
os.environ["WANDB_MODE"] = "offline"
os.environ["WANDB_API_KEY"] = "946e923f7717c88464dc01b43cdcb664b74b23b6"


In [19]:
from peft import LoraConfig, TaskType, get_peft_model, AutoPeftModelForCausalLM

model_name = "TinyLlama-new-1000"  # or any other base model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Add padding token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [20]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=4,  # rank
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj","v_proj"]  # adjust based on model
)

# Create PEFT model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 90,112 || all params: 1,100,138,496 || trainable%: 0.0082


In [21]:
from datasets import load_dataset, Dataset
from datasets import load_dataset
train_dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")
train_dataset = train_dataset.select(range(500))

# Format the data
data = []
for item in train_dataset:
    try:
        prompt = next((x["content"] for x in item["chosen"] if x["role"] == "user"), "")
        chosen = next((x["content"] for x in item["chosen"] if x["role"] == "assistant"), "")
        rejected = next((x["content"] for x in item["rejected"] if x["role"] == "assistant"), "")
        
        data.append({
            "prompt": prompt,
            "chosen": chosen,
            "rejected": rejected
        })
    except Exception:
        continue  # Skip entries that don't have the expected format

# Create HuggingFace dataset
formatted_dataset = Dataset.from_list(data)



# Preview
print(formatted_dataset)


README.md:   0%|          | 0.00/643 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/131M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/62135 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 500
})


In [28]:
from trl import DPOTrainer, DPOConfig

# Training arguments
training_args = DPOConfig(
    output_dir="./dpo-model",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=500,
    eval_steps=500,
    warmup_steps=20,
    beta=0.1,  # DPO beta parameter
    remove_unused_columns=False,
)

# Initialize DPO trainer
dpo_trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=formatted_dataset,
   processing_class=tokenizer,
    peft_config=peft_config,
)

Extracting prompt in train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Start training
dpo_trainer.train()

# Save the model
dpo_trainer.save_model("./final-dpo-model")

In [25]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
smoothie = SmoothingFunction().method4
# Dynamic device assignment
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# For generation, we'll load the model and use the `generate` method
model_for_generation = AutoModelForCausalLM.from_pretrained('final-dpo-model').to(device)
#model_for_generation = get_peft_model(model_for_generation, lora_config) # Ensure LoRA is applied

input_texts = [
    "So light is made up of colors?",
    "Why is the sky blue?",
    "How does gravity work?",
    "What causes rainbows?",
    "Why do we see lightning before thunder?",
    "How do plants make food?",
    "Why do we sleep?",
    "What is photosynthesis?",
    "How does the internet work?",
    "Why do birds migrate?"
]

reference_answers = [
    "Yes, light is made up of different colors. When they combine, they appear white. This can be separated with prisms, for example.",
    "The sky appears blue due to a phenomenon called Rayleigh scattering. Shorter blue wavelengths scatter more than red ones.",
    "Gravity is a force that attracts objects with mass toward each other. Earth pulls objects towards its center due to gravity.",
    "Rainbows occur when sunlight is both refracted and reflected inside raindrops, separating the light into different colors.",
    "Light travels faster than sound, so we see lightning before we hear thunder.",
    "Plants make food through photosynthesis, using sunlight, water, and carbon dioxide to produce energy and oxygen.",
    "We sleep to allow the body and brain to rest, recover, and process information from the day.",
    "Photosynthesis is the process plants use to convert sunlight into energy using chlorophyll, water, and carbon dioxide.",
    "The internet is a global network of computers that communicate via protocols like TCP/IP to exchange data.",
    "Birds migrate to find better food sources, breeding grounds, or climates that are more suitable during different seasons."
]

bleu_scores = []

for i in range(10):
    input_text = input_texts[i]
    reference = reference_answers[i]

    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    output = model_for_generation.generate(
        input_ids,
        max_length=70,
        num_return_sequences=1,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    candidate = generated_response.split()
    reference_tokens = [reference.split()]

    score = sentence_bleu(reference_tokens, candidate, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
    bleu_scores.append(score)

    print(f"\nPrompt {i+1}: {input_text}")
    print(f"Generated: {generated_response}")
    print(f"Reference: {reference}")
    print(f"BLEU-4 Score: {score:.4f}")

print("\nAverage BLEU-4 Score across all prompts: %.4f" % (sum(bleu_scores) / len(bleu_scores)))


Using device: cuda


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Prompt 1: So light is made up of colors?
Generated: So light is made up of colors?
Reference: Yes, light is made up of different colors. When they combine, they appear white. This can be separated with prisms, for example.
BLEU-4 Score: 0.0721

Prompt 2: Why is the sky blue?
Generated: Why is the sky blue?

1. The sun is the primary source of light in the sky.

2. The Earth's atmosphere absorbs and reflects some of the sun's light.

3. The Earth's rotation around the sun causes the sun's rays to travel in a circular path
Reference: The sky appears blue due to a phenomenon called Rayleigh scattering. Shorter blue wavelengths scatter more than red ones.
BLEU-4 Score: 0.0092

Prompt 3: How does gravity work?
Generated: How does gravity work?
Reference: Gravity is a force that attracts objects with mass toward each other. Earth pulls objects towards its center due to gravity.
BLEU-4 Score: 0.0000

Prompt 4: What causes rainbows?
Generated: What causes rainbows?
Reference: Rainbows occur w